In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO



def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        point = [x,y]
        print(point)


cv2.namedWindow('vid')
cv2.setMouseCallback('vid', RGB)


### Load yolo model
model = YOLO("models/yolo11n.pt")
names = model.names

### load the video source
cap = cv2.VideoCapture("media/vid_1.mp4")
count = 0

while True:
    # read frames if ret is true
    ret, frame = cap.read()
    if not ret:
        break
    
    # skip frames with odd numders
    count +=1
    if count%2 != 0:
        continue

    # resizing frames
    frame = cv2.resize(frame, (1020, 500))

    # run yolo on frames with persist tracking between frames
    results = model.track(frame, persist=True)
    
    #
    if results[0].boxes is not None and results[0].boxes.id is not None:
        bboxes = results[0].boxes.xyxy.int().cpu().tolist()
        class_ids = results[0].boxes.cls.int().cpu().tolist()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        confidences  =results[0].boxes.conf.cpu().tolist()

        for bbox, class_id, track_id, conf in zip(bboxes, class_ids, track_ids, confidences):
            class_name = names[class_id]
            x1, y1, x2, y2 = bbox

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(frame, f'{class_name}', (x1, y1-2), 1, 1, (0,255,0))

        cv2.imshow('vid', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()